In [8]:
import pandas as pd
import numpy as np
import torch.nn as nn
import torch
import numpy as np
import copy
import re
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from pickle import dump , load

In [12]:
class AdultClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden1 = nn.Linear(13, 64)
        self.act1 = nn.ReLU()
        self.hidden2 = nn.Linear(64, 128)
        self.act2 = nn.ReLU()
        self.hidden3 = nn.Linear(128, 32)
        self.act3 = nn.ReLU()
        self.hidden4 = nn.Linear(32, 16)
        self.act4 = nn.ReLU()
        self.output = nn.Linear(16, 1)
        self.act_output = nn.Sigmoid()
        self.dropout = nn.Dropout(0.1)

    def forward(self, x):
        x = self.dropout(self.act1(self.hidden1(x)))
        x = self.dropout(self.act2(self.hidden2(x)))
        x = self.dropout(self.act3(self.hidden3(x)))
        x = self.dropout(self.act4(self.hidden4(x)))
        # x = self.output(x)
        x = self.act_output(self.output(x))
        return x
    
    
def load_model():
    model = torch.load('model_training/adult_credit__model')
    model.eval()
    return model

def eval_model(model, input, scaler, columns_to_standardize):
    model.eval()
    # print(input)
    input = input.reshape(-1, 13)
    input = torch.tensor(input, dtype=torch.float32)
    input = input.numpy()
    input[:, columns_to_standardize] = scaler.transform(input[:, columns_to_standardize])
    input = torch.from_numpy(input).type(torch.float)
    with torch.no_grad():
        prob = model(input)
    return prob.tolist()[0][0]

def load_scaler(scaler_loc):
    return load(open(scaler_loc, 'rb'))

columns_to_standardize = [0, 1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12]

def load_data(data):
    return np.load(data)

data = pd.read_csv('output/NSGAIII_multi/0/final_gen.csv')

In [13]:
input = load_data('output/NSGAIII_multi/0/input_data.npy')
model = load_model()
scaler = load_scaler(scaler_loc='model_training/StandardScaler.pkl')
print(input)

[    17      0 332666      6      4      0      3      4      0      0
      0      4     39]


In [14]:
output = eval_model(model, input, scaler, columns_to_standardize)
print(output)

2.788897585093153e-11


In [15]:
data

,Unnamed: 0,Phenotype,o_1,o_2,o_3,o_4
0,0,x[12] += 3 ; x[3] += 8,1.000000,0.013558,0.153846,0.039456
1,1,x[3] += 7,1.000000,0.035897,0.076923,0.043811
2,2,x[3] += 5,1.000000,0.025641,0.076923,0.033555
3,3,x[3] += 7 ; x[2] *= 8000,0.999999,0.035125,0.153846,0.037649
4,4,x[3] += 6,1.000000,0.030769,0.076923,0.038683
...,...,...,...,...,...,...
452,452,x[2] += 300,1.000000,0.000023,0.076923,0.038254
453,453,x[3] += 3,1.000000,0.015385,0.076923,0.027828
454,454,x[9] *= 300,1.000000,0.000000,0.000000,0.038238
455,455,x[3] += 3,1.000000,0.015385,0.076923,0.027828


In [16]:
data = data.sort_values(by=['o_1'])
data = data[data['o_1'] < 0.45]
data.shape

(4, 6)

In [17]:
data.head()

,Unnamed: 0,Phenotype,o_1,o_2,o_3,o_4
456,456,x[2] -= 9000 ; x[9] += 300 ; x[9] *= 300,0.000000,0.077601,0.153846,0.114934
63,63,x[2] -= 300 ; x[6] += 1 ; x[5] -= 4,0.000000,0.077601,0.153846,0.114934
49,49,x[2] *= 300 ; x[11] -= 3,0.000000,0.077601,0.153846,0.114934
35,35,x[2] -= 9000 ; x[9] += 8000 ; x[3] += 9,0.306011,0.123755,0.230769,0.127290


In [19]:
def apply_phenotype(array, phenotype):
    # Split the phenotype string into individual operations
    operations = phenotype.split(';')
    x = copy.deepcopy(array)
    # Iterate over each operation and execute it
    for operation in operations:
        # Strip any leading/trailing whitespace from the operation
        operation = operation.strip()
        
        # Use the exec function to execute the operation on the array
        if operation:
            exec(operation)

    return x

In [20]:
data.head()

,Unnamed: 0,Phenotype,o_1,o_2,o_3,o_4
456,456,x[2] -= 9000 ; x[9] += 300 ; x[9] *= 300,0.000000,0.077601,0.153846,0.114934
63,63,x[2] -= 300 ; x[6] += 1 ; x[5] -= 4,0.000000,0.077601,0.153846,0.114934
49,49,x[2] *= 300 ; x[11] -= 3,0.000000,0.077601,0.153846,0.114934
35,35,x[2] -= 9000 ; x[9] += 8000 ; x[3] += 9,0.306011,0.123755,0.230769,0.127290


In [21]:
input_evolved = apply_phenotype(input, data['Phenotype'][456])
print(input_evolved)

[    17      0 323666      6      4      0      3      4      0  90000
      0      4     39]


In [22]:
output = eval_model(model, input_evolved, scaler, columns_to_standardize)
print(output)

1.0
